<a href="https://colab.research.google.com/github/Divya2610/ANN-Classification-Churn/blob/main/Another_copy_of_gemma_3_1b_it.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U transformers

## Local Inference on GPU
Model page: https://huggingface.co/google/gemma-3-1b-it

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/google/gemma-3-1b-it)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

The model you are trying to use is gated. Please make sure you have access to it by visiting the model page.To run inference, either set HF_TOKEN in your environment variables/ Secrets or run the following cell to login. 🤗

In [ ]:
pip install fastapi uvicorn transformers torch sentence-transformers faiss-cpu accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 105.2 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login
login(new_session=False)

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="google/gemma-3-1b-it")
messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe(messages)

config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

Device set to use cuda:0


[{'generated_text': [{'role': 'user', 'content': 'Who are you?'},
   {'role': 'assistant',
    'content': 'Hi there! I’m Gemma, a large language model created by the Gemma team at Google DeepMind. I’m an open-weights model, which means I’m publicly available for anyone to use! \n\nI’m designed to take text and images as input and produce text as output. \n\nIt’s nice to meet you! 😊'}]}]

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-1b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-3-1b-it")
messages = [
    {"role": "user", "content": "Who are you?"},
]
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=40)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

Hi there! I’m Gemma, a large language model created by the Gemma team at Google DeepMind. I’m an open-weights model, which means I’m publicly available for use


In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

app = FastAPI()

tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-1b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-3-1b-it",
    torch_dtype=torch.bfloat16,
    device_map="auto"
).eval()

class Prompt(BaseModel):
    text: str

@app.post("/generate")
def generate(prompt: Prompt):
    messages = [{"role": "user", "content": prompt.text}]
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    with torch.inference_mode():
        outputs = model.generate(inputs, max_new_tokens=80)

    result = tokenizer.decode(
        outputs[0][inputs.shape[-1]:],
        skip_special_tokens=True
    )
    return {"response": result}


`torch_dtype` is deprecated! Use `dtype` instead!


In [ ]:
!huggingface-cli login


⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `hf auth whoami` to get more information or `hf auth logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): Traceb

In [ ]:
mkdir llm_api

mkdir: cannot create directory ‘llm_api’: File exists


In [ ]:
cd llm_api

/content/llm_api


In [ ]:
!touch app.py

In [ ]:
!ls /content


app.py	drive  llm_api	sample_data


In [ ]:
%%writefile /content/llm_api/app.py
from fastapi import FastAPI
from pydantic import BaseModel
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

MODEL_NAME = "google/gemma-3-1b-it"

app = FastAPI()

tokenizer = None
model = None

class Prompt(BaseModel):
    text: str

@app.get("/")
def root():
    return {"status": "ok", "message": "LLM API is running"}

@app.on_event("startup")
def load_model():
    global tokenizer, model
    print("🔄 Loading model...")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        dtype=torch.float16,      # 🔧 small fix (torch_dtype → dtype)
        device_map="auto",
    ).eval()
    print("✅ Model loaded")

@app.post("/generate")
def generate(prompt: Prompt):
    messages = [{"role": "user", "content": prompt.text}]
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to(model.device)

    with torch.inference_mode():
        outputs = model.generate(inputs, max_new_tokens=80)

    result = tokenizer.decode(
        outputs[0][inputs.shape[-1]:],
        skip_special_tokens=True,
    )
    return {"response": result}

Overwriting /content/llm_api/app.py


In [ ]:
!sed -n '1,50p' /content/llm_api/app.py


from fastapi import FastAPI
from pydantic import BaseModel
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

MODEL_NAME = "google/gemma-3-1b-it"

app = FastAPI()

tokenizer = None
model = None

class Prompt(BaseModel):
    text: str

@app.get("/")
def root():
    return {"status": "ok", "message": "LLM API is running"}

@app.on_event("startup")
def load_model():
    global tokenizer, model
    print("🔄 Loading model...")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        dtype=torch.float16,      # 🔧 small fix (torch_dtype → dtype)
        device_map="auto",
    ).eval()
    print("✅ Model loaded")

@app.post("/generate")
def generate(prompt: Prompt):
    messages = [{"role": "user", "content": prompt.text}]
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to(model.device)

    with torch.infere

In [ ]:
!ls /content/llm_api


app.py	__init__.py  __pycache__


In [ ]:
import os
import sys

os.chdir('/content') # Change the current directory back to /content

# Remove the module from sys.modules to force a reload
if 'llm_api.app' in sys.modules:
    del sys.modules['llm_api.app']
if 'llm_api' in sys.modules:
    del sys.modules['llm_api']

from llm_api.app import app
print(type(app))

<class 'fastapi.applications.FastAPI'>


In [ ]:
!nohup python -m uvicorn llm_api.app:app --host 0.0.0.0 --port 8000 > uvicorn.log 2>&1 &

In [ ]:
import requests

requests.get("http://localhost:8000/").json()


{'status': 'ok', 'message': 'LLM API is running'}

First, let's stop any processes currently listening on port 8000 to ensure a clean restart. We'll use `lsof` to find the process ID (PID) and `kill` to terminate it.

In [ ]:
!kill $(lsof -t -i:8000)

Now, let's restart the Uvicorn server with the updated `app.py` which includes the new root endpoint.

In [ ]:
!nohup python -m uvicorn llm_api.app:app --host 0.0.0.0 --port 8000 > uvicorn.log 2>&1 &

Finally, let's try calling the root endpoint again to verify the changes.

In [ ]:
import requests
import time

time.sleep(30) # Wait for 30 seconds to allow the server and model to fully start
requests.get("http://localhost:8000/").json()

{'status': 'ok', 'message': 'LLM API is running'}

In [ ]:
!cat uvicorn.log

INFO:     Started server process [26559]
INFO:     Waiting for application startup.
2026-01-12 09:36:43.012893: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768210603.048919   26559 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768210603.060557   26559 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768210603.086881   26559 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768210603.086909   26559 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W00

Now that the server is running, let's test the text generation endpoint by sending a POST request with a sample prompt.

In [ ]:
import requests

response = requests.post(
    "http://localhost:8000/generate",
    json={"text": "What is the capital of France?"}
)
print(response.json())

{'response': 'The capital of France is **Paris**. \n\nWould you like to know more about Paris? For example, I could tell you about its history, landmarks, or things to do there!'}


Now that the server is running, let's test the text generation endpoint by sending a POST request with a sample prompt.

In [ ]:
import requests

response = requests.post(
    "http://localhost:8000/generate",
    json={"text": "What is the capital of Japan?"}
)
print(response.json())

{'response': 'The capital of Japan is **Tokyo**.\n'}


# Task
Enhance the `app.py` FastAPI application to support Retrieval Augmented Generation (RAG). This involves installing necessary RAG dependencies, implementing functions for loading and processing documents into chunks, generating vector embeddings for these chunks using `sentence-transformers`, and building a persistent FAISS vector store for efficient retrieval.

## Prepare RAG Dependencies

### Subtask:
Ensure all necessary Python libraries for RAG (Retrieval Augmented Generation) are installed, such as `langchain` or similar frameworks, alongside the existing `transformers`, `sentence-transformers`, and `faiss-cpu`.


**Reasoning**:
Install `langchain` as specified in the instructions to ensure all necessary libraries for RAG are present.



In [ ]:
pip install langchain

**Reasoning**:
Install `tiktoken` as specified in the instructions to ensure all necessary libraries for RAG are present, especially for Langchain's tokenization needs.



In [ ]:
pip install tiktoken

## Implement Document Processing

### Subtask:
Develop Python functions within the `app.py` that can load a given document, split it into manageable chunks, and generate vector embeddings for each chunk using `sentence-transformers`.


**Reasoning**:
The subtask requires modifying the `app.py` file to include RAG functionalities. This step involves adding necessary imports, initializing the `SentenceTransformer` model, and defining functions for loading/chunking documents and generating embeddings, all within the `app.py` file. I will use `%%writefile` to update the file with the new code.



In [ ]:
%%writefile /content/llm_api/app.py
from fastapi import FastAPI
from pydantic import BaseModel
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer

MODEL_NAME = "google/gemma-3-1b-it"
EMBEDDING_MODEL_NAME = "all-MiniLM-L6-v2"

app = FastAPI()

tokenizer = None
model = None
embedding_model = None

class Prompt(BaseModel):
    text: str

@app.get("/")
def root():
    return {"status": "ok", "message": "LLM API is running"}

@app.on_event("startup")
def load_model():
    global tokenizer, model, embedding_model
    print(" Loading models...")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        dtype=torch.float16,
        device_map="auto",
    ).eval()
    embedding_model = SentenceTransformer(EMBEDDING_MODEL_NAME)
    print(" Models loaded")

def load_and_chunk_document(file_path: str):
    loader = TextLoader(file_path)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.split_documents(documents)
    return chunks

def generate_embeddings(chunks):
    texts = [doc.page_content for doc in chunks]
    embeddings = embedding_model.encode(texts, convert_to_tensor=True)
    return embeddings

@app.post("/generate")
def generate(prompt: Prompt):
    messages = [{"role": "user", "content": prompt.text}]
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to(model.device)

    with torch.inference_mode():
        outputs = model.generate(inputs, max_new_tokens=80)

    result = tokenizer.decode(
        outputs[0][inputs.shape[-1]:],
        skip_special_tokens=True,
    )
    return {"response": result}

Overwriting /content/llm_api/app.py


## Build and Persist FAISS Vector Store

### Subtask:
Create a FAISS (Facebook AI Similarity Search) index from the generated document embeddings. This index will serve as our searchable knowledge base. Implement functionality to save and load this index for persistence.


**Reasoning**:
The subtask requires modifying the `app.py` file to include FAISS vector store functionalities. This step involves adding necessary imports, defining constants, updating the `load_model` function to properly initialize `embedding_model` for Langchain's FAISS, adding a global `vector_store` variable, and implementing `create_and_save_vector_store` and `load_vector_store` functions, along with calling `load_vector_store` during startup. I will use `%%writefile` to update the file with the new code.



In [ ]:
%%writefile /content/llm_api/app.py
from fastapi import FastAPI
from pydantic import BaseModel
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import SentenceTransformerEmbeddings
import os

MODEL_NAME = "google/gemma-3-1b-it"
EMBEDDING_MODEL_NAME = "all-MiniLM-L6-v2"
FAISS_INDEX_PATH = "faiss_index"

app = FastAPI()

tokenizer = None
model = None
embedding_model_langchain = None # Changed name to avoid conflict and signify Langchain wrapper
vector_store = None

class Prompt(BaseModel):
    text: str

@app.get("/")
def root():
    return {"status": "ok", "message": "LLM API is running"}

@app.on_event("startup")
def load_model():
    global tokenizer, model, embedding_model_langchain, vector_store
    print("🔄 Loading models...")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        dtype=torch.float16,
        device_map="auto",
    ).eval()

    # Initialize SentenceTransformer and wrap it for Langchain compatibility
    st_model = SentenceTransformer(EMBEDDING_MODEL_NAME)
    embedding_model_langchain = SentenceTransformerEmbeddings(model_name=EMBEDDING_MODEL_NAME, model_kwargs={'device': 'cuda'}) # Using 'cuda' if available
    print("✅ Models loaded")

    # Attempt to load existing vector store on startup
    load_vector_store()

def load_and_chunk_document(file_path: str):
    loader = TextLoader(file_path)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.split_documents(documents)
    return chunks

def create_and_save_vector_store(chunks):
    global vector_store
    print("Building and saving FAISS index...")
    vector_store = FAISS.from_documents(chunks, embedding_model_langchain)
    vector_store.save_local(FAISS_INDEX_PATH)
    print(f"FAISS index saved to {FAISS_INDEX_PATH}")

def load_vector_store():
    global vector_store
    if os.path.exists(FAISS_INDEX_PATH):
        print(f"Loading FAISS index from {FAISS_INDEX_PATH}...")
        # allow_dangerous_deserialization is required for loading FAISS index
        vector_store = FAISS.load_local(FAISS_INDEX_PATH, embedding_model_langchain, allow_dangerous_deserialization=True)
        print("FAISS index loaded.")
    else:
        print("No existing FAISS index found. A new one will be created if needed.")

@app.post("/generate")
def generate(prompt: Prompt):
    messages = [{"role": "user", "content": prompt.text}]
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to(model.device)

    with torch.inference_mode():
        outputs = model.generate(inputs, max_new_tokens=80)

    result = tokenizer.decode(
        outputs[0][inputs.shape[-1]:],
        skip_special_tokens=True,
    )
    return {"response": result}

Overwriting /content/llm_api/app.py


## Develop Retrieval Mechanism

### Subtask:
Write a function to perform a similarity search on the FAISS index given a user query, retrieving the most relevant document chunks.


**Reasoning**:
The subtask requires adding a `retrieve_documents` function to `app.py` that performs a similarity search on the `vector_store`. I will use `%%writefile` to update the file, ensuring the new function is included.



In [ ]:
%%writefile /content/llm_api/app.py
from fastapi import FastAPI
from pydantic import BaseModel
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import SentenceTransformerEmbeddings
import os

MODEL_NAME = "google/gemma-3-1b-it"
EMBEDDING_MODEL_NAME = "all-MiniLM-L6-v2"
FAISS_INDEX_PATH = "faiss_index"

app = FastAPI()

tokenizer = None
model = None
embedding_model_langchain = None
vector_store = None

class Prompt(BaseModel):
    text: str

@app.get("/")
def root():
    return {"status": "ok", "message": "LLM API is running"}

@app.on_event("startup")
def load_model():
    global tokenizer, model, embedding_model_langchain, vector_store
    print("🔄 Loading models...")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        dtype=torch.float16,
        device_map="auto",
    ).eval()

    # Initialize SentenceTransformer and wrap it for Langchain compatibility
    # st_model = SentenceTransformer(EMBEDDING_MODEL_NAME) # This line is not needed if using SentenceTransformerEmbeddings
    embedding_model_langchain = SentenceTransformerEmbeddings(model_name=EMBEDDING_MODEL_NAME, model_kwargs={'device': 'cuda'})
    print("✅ Models loaded")

    # Attempt to load existing vector store on startup
    load_vector_store()

def load_and_chunk_document(file_path: str):
    loader = TextLoader(file_path)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.split_documents(documents)
    return chunks

def create_and_save_vector_store(chunks):
    global vector_store
    print("Building and saving FAISS index...")
    vector_store = FAISS.from_documents(chunks, embedding_model_langchain)
    vector_store.save_local(FAISS_INDEX_PATH)
    print(f"FAISS index saved to {FAISS_INDEX_PATH}")

def load_vector_store():
    global vector_store
    if os.path.exists(FAISS_INDEX_PATH):
        print(f"Loading FAISS index from {FAISS_INDEX_PATH}...")
        vector_store = FAISS.load_local(FAISS_INDEX_PATH, embedding_model_langchain, allow_dangerous_deserialization=True)
        print("FAISS index loaded.")
    else:
        print("No existing FAISS index found. A new one will be created if needed.")

def retrieve_documents(query: str, k: int = 4):
    """Performs a similarity search on the FAISS index to retrieve relevant document chunks."""
    if vector_store is None:
        print("Vector store not initialized. Cannot retrieve documents.")
        return []
    print(f"Retrieving top {k} documents for query: '{query}'")
    docs = vector_store.similarity_search(query, k=k)
    return docs

@app.post("/generate")
def generate(prompt: Prompt):
    messages = [{"role": "user", "content": prompt.text}]
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to(model.device)

    with torch.inference_mode():
        outputs = model.generate(inputs, max_new_tokens=80)

    result = tokenizer.decode(
        outputs[0][inputs.shape[-1]:],
        skip_special_tokens=True,
    )
    return {"response": result}

Overwriting /content/llm_api/app.py


## Integrate RAG with Language Model

### Subtask:
Modify the text generation logic to incorporate the retrieved document chunks as context for the language model. This will enable the model to answer questions based on the document content and generate summaries.


**Reasoning**:
I need to update the `generate` function in `app.py` to implement the RAG logic as specified, including an optional `context_enabled` parameter, retrieving documents, formatting context, and constructing a new prompt. This requires overwriting the existing `app.py` file.



In [ ]:
%%writefile /content/llm_api/app.py
from fastapi import FastAPI
from pydantic import BaseModel
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import SentenceTransformerEmbeddings
import os

MODEL_NAME = "google/gemma-3-1b-it"
EMBEDDING_MODEL_NAME = "all-MiniLM-L6-v2"
FAISS_INDEX_PATH = "faiss_index"

app = FastAPI()

tokenizer = None
model = None
embedding_model_langchain = None
vector_store = None

class Prompt(BaseModel):
    text: str
    context_enabled: bool = False # New parameter

@app.get("/")
def root():
    return {"status": "ok", "message": "LLM API is running"}

@app.on_event("startup")
def load_model():
    global tokenizer, model, embedding_model_langchain, vector_store
    print("🔄 Loading models...")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        dtype=torch.float16,
        device_map="auto",
    ).eval()

    embedding_model_langchain = SentenceTransformerEmbeddings(model_name=EMBEDDING_MODEL_NAME, model_kwargs={'device': 'cuda'})
    print("✅ Models loaded")

    load_vector_store()

def load_and_chunk_document(file_path: str):
    loader = TextLoader(file_path)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.split_documents(documents)
    return chunks

def create_and_save_vector_store(chunks):
    global vector_store
    print("Building and saving FAISS index...")
    vector_store = FAISS.from_documents(chunks, embedding_model_langchain)
    vector_store.save_local(FAISS_INDEX_PATH)
    print(f"FAISS index saved to {FAISS_INDEX_PATH}")

def load_vector_store():
    global vector_store
    if os.path.exists(FAISS_INDEX_PATH):
        print(f"Loading FAISS index from {FAISS_INDEX_PATH}...")
        vector_store = FAISS.load_local(FAISS_INDEX_PATH, embedding_model_langchain, allow_dangerous_deserialization=True)
        print("FAISS index loaded.")
    else:
        print("No existing FAISS index found. A new one will be created if needed.")

def retrieve_documents(query: str, k: int = 4):
    if vector_store is None:
        print("Vector store not initialized. Cannot retrieve documents.")
        return []
    print(f"Retrieving top {k} documents for query: '{query}'")
    docs = vector_store.similarity_search(query, k=k)
    return docs

@app.post("/generate")
def generate(prompt: Prompt):
    user_query = prompt.text
    if prompt.context_enabled:
        # Retrieve relevant documents
        retrieved_docs = retrieve_documents(user_query)
        if retrieved_docs:
            # Format retrieved documents into a context string
            context = "\n\n".join([doc.page_content for doc in retrieved_docs])
            # Construct a new prompt with context
            messages = [
                {"role": "user", "content": f"Based on the following information:\n\n{context}\n\nAnswer the question: {user_query}"}
            ]
        else:
            messages = [{"role": "user", "content": user_query}]
    else:
        messages = [{"role": "user", "content": user_query}]

    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to(model.device)

    with torch.inference_mode():
        outputs = model.generate(inputs, max_new_tokens=80)

    result = tokenizer.decode(
        outputs[0][inputs.shape[-1]:],
        skip_special_tokens=True,
    )
    return {"response": result}

Overwriting /content/llm_api/app.py


**Reasoning**:
Now that `app.py` has been updated with the RAG logic, I will display the first 50 lines of the file to verify the changes, especially the `generate` function and the new `context_enabled` parameter in the `Prompt` class.



In [ ]:
%%bash
sed -n '1,100p' /content/llm_api/app.py

from fastapi import FastAPI
from pydantic import BaseModel
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import SentenceTransformerEmbeddings
import os

MODEL_NAME = "google/gemma-3-1b-it"
EMBEDDING_MODEL_NAME = "all-MiniLM-L6-v2"
FAISS_INDEX_PATH = "faiss_index"

app = FastAPI()

tokenizer = None
model = None
embedding_model_langchain = None
vector_store = None

class Prompt(BaseModel):
    text: str
    context_enabled: bool = False # New parameter

@app.get("/")
def root():
    return {"status": "ok", "message": "LLM API is running"}

@app.on_event("startup")
def load_model():
    global tokenizer, model, embedding_model_langchain, vector_store
    print("🔄 Loading models...")
    tokenizer =

## Update FastAPI for RAG Endpoints

### Subtask:
Add new endpoints to the FastAPI application (in `app.py`) for: 1) uploading and processing a document (which will trigger vector store creation), 2) requesting a summary of the loaded document, and 3) asking questions related to the document.


**Reasoning**:
I need to update the `app.py` file to include the new RAG endpoints for document upload, RAG-based questioning, and summarization, as well as the necessary imports. I will use `%%writefile` to overwrite the entire file with the updated content.



In [ ]:
%%writefile /content/llm_api/app.py
from fastapi import FastAPI, UploadFile, File
from pydantic import BaseModel
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import SentenceTransformerEmbeddings
import os
import shutil

MODEL_NAME = "google/gemma-3-1b-it"
EMBEDDING_MODEL_NAME = "all-MiniLM-L6-v2"
FAISS_INDEX_PATH = "faiss_index"

app = FastAPI()

tokenizer = None
model = None
embedding_model_langchain = None
vector_store = None

class Prompt(BaseModel):
    text: str

@app.get("/")
def root():
    return {"status": "ok", "message": "LLM API is running"}

@app.on_event("startup")
def load_model():
    global tokenizer, model, embedding_model_langchain, vector_store
    print("🔄 Loading models...")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        dtype=torch.float16,
        device_map="auto",
    ).eval()

    embedding_model_langchain = SentenceTransformerEmbeddings(model_name=EMBEDDING_MODEL_NAME, model_kwargs={'device': 'cuda'})
    print("✅ Models loaded")

    load_vector_store()

def load_and_chunk_document(file_path: str):
    loader = TextLoader(file_path)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.split_documents(documents)
    return chunks

def create_and_save_vector_store(chunks):
    global vector_store
    print("Building and saving FAISS index...")
    vector_store = FAISS.from_documents(chunks, embedding_model_langchain)
    vector_store.save_local(FAISS_INDEX_PATH)
    print(f"FAISS index saved to {FAISS_INDEX_PATH}")

def load_vector_store():
    global vector_store
    if os.path.exists(FAISS_INDEX_PATH):
        print(f"Loading FAISS index from {FAISS_INDEX_PATH}...")
        vector_store = FAISS.load_local(FAISS_INDEX_PATH, embedding_model_langchain, allow_dangerous_deserialization=True)
        print("FAISS index loaded.")
    else:
        print("No existing FAISS index found. A new one will be created if needed.")

def retrieve_documents(query: str, k: int = 4):
    if vector_store is None:
        print("Vector store not initialized. Cannot retrieve documents.")
        return []
    print(f"Retrieving top {k} documents for query: '{query}'")
    docs = vector_store.similarity_search(query, k=k)
    return docs

@app.post("/upload-document")
async def upload_document(file: UploadFile = File(...)):
    try:
        # Create a temporary file to store the uploaded content
        temp_file_path = f"/tmp/{file.filename}"
        with open(temp_file_path, "wb") as buffer:
            shutil.copyfileobj(file.file, buffer)

        # Process the document and create the vector store
        chunks = load_and_chunk_document(temp_file_path)
        create_and_save_vector_store(chunks)

        return {"message": f"Document '{file.filename}' processed and vector store built successfully."}
    except Exception as e:
        return {"error": str(e)}
    finally:
        # Clean up the temporary file
        if os.path.exists(temp_file_path):
            os.remove(temp_file_path)

@app.post("/ask-rag")
def ask_rag(prompt: Prompt):
    user_query = prompt.text
    retrieved_docs = retrieve_documents(user_query)

    if retrieved_docs:
        context = "\n\n".join([doc.page_content for doc in retrieved_docs])
        messages = [
            {"role": "user", "content": f"Based on the following information:\n\n{context}\n\nAnswer the question: {user_query}"}
        ]
    else:
        messages = [
            {"role": "user", "content": f"I could not find relevant information in the knowledge base. Please try rephrasing or asking a general question: {user_query}"}
        ]

    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to(model.device)

    with torch.inference_mode():
        outputs = model.generate(inputs, max_new_tokens=80)

    result = tokenizer.decode(
        outputs[0][inputs.shape[-1]:],
        skip_special_tokens=True,
    )
    return {"response": result}

@app.post("/summarize-rag")
def summarize_rag(prompt: Prompt):
    summary_topic = prompt.text if prompt.text else "the provided documents"
    retrieval_query = f"Information about {summary_topic}"

    retrieved_docs = retrieve_documents(retrieval_query)

    if retrieved_docs:
        context = "\n\n".join([doc.page_content for doc in retrieved_docs])
        messages = [
            {"role": "user", "content": f"Based on the following information:\n\n{context}\n\nSummarize this information in a concise way."}
        ]
    else:
        messages = [
            {"role": "user", "content": f"I could not find relevant information to summarize about {summary_topic}."}
        ]

    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to(model.device)

    with torch.inference_mode():
        outputs = model.generate(inputs, max_new_tokens=150) # Increased max_new_tokens for summaries

    result = tokenizer.decode(
        outputs[0][inputs.shape[-1]:],
        skip_special_tokens=True,
    )
    return {"summary": result}

@app.post("/generate")
def generate(prompt: Prompt):
    messages = [
        {"role": "user", "content": prompt.text}
    ]

    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to(model.device)

    with torch.inference_mode():
        outputs = model.generate(inputs, max_new_tokens=80)

    result = tokenizer.decode(
        outputs[0][inputs.shape[-1]:],
        skip_special_tokens=True,
    )
    return {"response": result}

Overwriting /content/llm_api/app.py


**Reasoning**:
The `app.py` file has been updated with the new RAG endpoints. Now I will display the first 100 lines of the file to verify that the changes have been applied correctly, especially the new endpoints and imports.



In [ ]:
%%bash
sed -n '1,150p' /content/llm_api/app.py

from fastapi import FastAPI, UploadFile, File
from pydantic import BaseModel
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import SentenceTransformerEmbeddings
import os
import shutil

MODEL_NAME = "google/gemma-3-1b-it"
EMBEDDING_MODEL_NAME = "all-MiniLM-L6-v2"
FAISS_INDEX_PATH = "faiss_index"

app = FastAPI()

tokenizer = None
model = None
embedding_model_langchain = None
vector_store = None

class Prompt(BaseModel):
    text: str

@app.get("/")
def root():
    return {"status": "ok", "message": "LLM API is running"}

@app.on_event("startup")
def load_model():
    global tokenizer, model, embedding_model_langchain, vector_store
    print("🔄 Loading models...")
    tokenizer = AutoTokenizer.fro

## Test RAG Functionality

### Subtask:
Thoroughly test all new RAG-related FastAPI endpoints (document upload, summarization, and question answering) to ensure they are working as expected and providing accurate results.


**Reasoning**:
The first step in testing the RAG functionality is to stop the currently running Uvicorn server to ensure a clean restart with the updated `app.py`.



In [ ]:
!kill $(lsof -t -i:8000)

**Reasoning**:
Now that the previous Uvicorn server has been stopped, I need to restart it to load the updated `app.py` with the new RAG endpoints.



In [ ]:
!nohup python -m uvicorn llm_api.app:app --host 0.0.0.0 --port 8000 > uvicorn.log 2>&1 &

**Reasoning**:
Now that the previous Uvicorn server has been stopped, I need to restart it to load the updated `app.py` with the new RAG endpoints.



In [ ]:
!nohup python -m uvicorn llm_api.app:app --host 0.0.0.0 --port 8000 > uvicorn.log 2>&1 &

**Reasoning**:
Now that the previous Uvicorn server has been stopped, I need to restart it to load the updated `app.py` with the new RAG endpoints.



In [ ]:
import time
time.sleep(5) # Give the server a moment to shut down cleanly before restarting
!nohup python -m uvicorn llm_api.app:app --host 0.0.0.0 --port 8000 > uvicorn.log 2>&1 &

**Reasoning**:
As instructed, I will create a dummy text file named `test_document.txt` with sample content in the `/content/llm_api/` directory to be used for testing the RAG functionalities.



In [ ]:
%%writefile /content/llm_api/test_document.txt
Jupiter is the largest planet in our solar system. It is a gas giant with a mass more than two and a half times that of all the other planets in the Solar System combined, but slightly less than one-thousandth the mass of the Sun. Jupiter is the fifth planet from the Sun. It is primarily composed of hydrogen and helium. It has a Great Red Spot, which is a giant storm larger than Earth. Jupiter has 79 known moons, including the four largest: Io, Europa, Ganymede, and Callisto, often referred to as the Galilean moons.

Writing /content/llm_api/test_document.txt


**Reasoning**:
I need to upload the `test_document.txt` file to the `/upload-document` endpoint using `requests.post`. I will ensure the file is sent correctly as `multipart/form-data` as per the instructions.



In [ ]:
import requests
import os

file_path = "/content/llm_api/test_document.txt"

with open(file_path, "rb") as f:
    files = {'file': (os.path.basename(file_path), f, 'text/plain')}
    response = requests.post("http://localhost:8000/upload-document", files=files)

print(response.json())

ConnectionError: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /upload-document (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7eaa71c0f7d0>: Failed to establish a new connection: [Errno 111] Connection refused'))

**Reasoning**:
The previous attempt to upload the document failed with a `ConnectionError` because the FastAPI server was likely not fully started or accessible. I need to add a sufficient delay after restarting the Uvicorn server to ensure all models are loaded and the server is ready to handle requests before attempting to upload the document.



In [ ]:
import requests
import os
import time
import subprocess # Added import for subprocess

# Ensure any previous Uvicorn processes are killed
# Use a more robust kill command that doesn't error if no process is found
subprocess.run("kill -9 $(lsof -t -i:8000) 2>/dev/null || true", shell=True)

# Restart Uvicorn server with the updated app.py
uvicorn_log_path = "/content/uvicorn.log"
# Clear the log file before starting uvicorn to avoid reading old logs
open(uvicorn_log_path, 'w').close()
# Use subprocess.Popen for non-blocking execution of uvicorn
subprocess.Popen(f"nohup python -m uvicorn llm_api.app:app --host 0.0.0.0 --port 8000 > {uvicorn_log_path} 2>&1 &", shell=True)

# Wait for the server to start up and models to load (polling log file)
print("Waiting for FastAPI server to start and models to load...")
server_and_models_ready = False
start_time = time.time()
timeout = 240 # Increased timeout to 4 minutes for model loading

while time.time() - start_time < timeout:
    if os.path.exists(uvicorn_log_path):
        with open(uvicorn_log_path, 'r') as f:
            log_content = f.read()
            if "✅ Models loaded" in log_content:
                server_and_models_ready = True
                print("FastAPI server and models are ready.")
                break
    time.sleep(5) # Check every 5 seconds

if not server_and_models_ready:
    # Read the log file one last time if not ready, to include it in the error message
    final_log_content = "Log file not found or empty." if not os.path.exists(uvicorn_log_path) else open(uvicorn_log_path, 'r').read()
    raise Exception(f"FastAPI server and models did not start in time ({timeout} seconds). Check {uvicorn_log_path} for errors. \n--- Uvicorn Log Content ---\n{final_log_content}\n---------------------------")

file_path = "/content/llm_api/test_document.txt"

with open(file_path, "rb") as f:
    files = {'file': (os.path.basename(file_path), f, 'text/plain')}
    response = requests.post("http://localhost:8000/upload-document", files=files)

print(response.json())

Waiting for FastAPI server to start and models to load...


Exception: FastAPI server and models did not start in time (240 seconds). Check /content/uvicorn.log for errors. 
--- Uvicorn Log Content ---
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/__main__.py", line 4, in <module>
    uvicorn.main()
  File "/usr/local/lib/python3.12/dist-packages/click/core.py", line 1485, in __call__
    return self.main(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/click/core.py", line 1406, in main
    rv = self.invoke(ctx)
         ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/click/core.py", line 1269, in invoke
    return ctx.invoke(self.callback, **ctx.params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/click/core.py", line 824, in invoke
    return callback(*args, **kwargs)
   

**Reasoning**:
The previous error `ModuleNotFoundError: No module named 'langchain_community'` indicates that the `langchain-community` package, which provides `TextLoader`, is missing. Although `langchain` was installed, its sub-packages might not be correctly recognized or installed. Explicitly installing `langchain-community` will resolve this dependency issue.



In [ ]:
pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.2/490.2 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.2.1
    Uninstalling langchain-core-1.2.1:
      Successfully uninstalled langchain-core-1.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is inc

**Reasoning**:
The `langchain-community` package has now been installed, which should resolve the `ModuleNotFoundError`. I will now re-attempt to restart the FastAPI server and upload the document, using the robust polling mechanism to ensure the server and models are fully loaded before proceeding.



In [ ]:
import requests
import os
import time
import subprocess # Added import for subprocess

# Ensure any previous Uvicorn processes are killed
# Use a more robust kill command that doesn't error if no process is found
subprocess.run("kill -9 $(lsof -t -i:8000) 2>/dev/null || true", shell=True)

# Restart Uvicorn server with the updated app.py
uvicorn_log_path = "/content/uvicorn.log"
# Clear the log file before starting uvicorn to avoid reading old logs
open(uvicorn_log_path, 'w').close()
# Use subprocess.Popen for non-blocking execution of uvicorn
subprocess.Popen(f"nohup python -m uvicorn llm_api.app:app --host 0.0.0.0 --port 8000 > {uvicorn_log_path} 2>&1 &", shell=True)

# Wait for the server to start up and models to load (polling log file)
print("Waiting for FastAPI server to start and models to load...")
server_and_models_ready = False
start_time = time.time()
timeout = 240 # Increased timeout to 4 minutes for model loading

while time.time() - start_time < timeout:
    if os.path.exists(uvicorn_log_path):
        with open(uvicorn_log_path, 'r') as f:
            log_content = f.read()
            if "✅ Models loaded" in log_content:
                server_and_models_ready = True
                print("FastAPI server and models are ready.")
                break
    time.sleep(5) # Check every 5 seconds

if not server_and_models_ready:
    # Read the log file one last time if not ready, to include it in the error message
    final_log_content = "Log file not found or empty." if not os.path.exists(uvicorn_log_path) else open(uvicorn_log_path, 'r').read()
    raise Exception(f"FastAPI server and models did not start in time ({timeout} seconds). Check {uvicorn_log_path} for errors. \n--- Uvicorn Log Content ---\n{final_log_content}\n---------------------------")

file_path = "/content/llm_api/test_document.txt"

with open(file_path, "rb") as f:
    files = {'file': (os.path.basename(file_path), f, 'text/plain')}
    response = requests.post("http://localhost:8000/upload-document", files=files)

print(response.json())

**Reasoning**:
Now that the server is confirmed to be running and the document has been uploaded, I will test the `/ask-rag` endpoint by sending a POST request with a question related to the content of `test_document.txt` to verify the RAG functionality.



In [ ]:
import requests
import os
import time
import subprocess

# Ensure any previous Uvicorn processes are killed
subprocess.run("kill -9 $(lsof -t -i:8000) 2>/dev/null || true", shell=True)

# Change directory to where app.py is located
os.chdir('/content/llm_api')

# Restart Uvicorn server with the updated app.py
uvicorn_log_path = "uvicorn.log" # Log file will be in /content/llm_api
# Clear the log file before starting uvicorn to avoid reading old logs
open(uvicorn_log_path, 'w').close()
# Use subprocess.Popen for non-blocking execution of uvicorn
subprocess.Popen(f"nohup python -m uvicorn app:app --host 0.0.0.0 --port 8000 > {uvicorn_log_path} 2>&1 &", shell=True)

# Wait for the server to start up and models to load (polling log file)
print("Waiting for FastAPI server to start and models to load...")
server_and_models_ready = False
start_time = time.time()
timeout = 300 # Increased timeout to 5 minutes for model loading

while time.time() - start_time < timeout:
    if os.path.exists(uvicorn_log_path):
        with open(uvicorn_log_path, 'r') as f:
            log_content = f.read()
            if "✅ Models loaded" in log_content:
                server_and_models_ready = True
                print("FastAPI server and models are ready.")
                break
    time.sleep(5) # Check every 5 seconds

if not server_and_models_ready:
    # Read the log file one last time if not ready, to include it in the error message
    final_log_content = "Log file not found or empty." if not os.path.exists(uvicorn_log_path) else open(uvicorn_log_path, 'r').read()
    raise Exception(f"FastAPI server and models did not start in time ({timeout} seconds). Check {uvicorn_log_path} for errors. \n--- Uvicorn Log Content ---\n{final_log_content}\n---------------------------")

# Create a dummy text file named test_document.txt
document_content = """
Jupiter is the largest planet in our solar system. It is a gas giant with a mass more than two and a half times that of all the other planets in the Solar System combined, but slightly less than one-thousandth the mass of the Sun. Jupiter is the fifth planet from the Sun. It is primarily composed of hydrogen and helium. It has a Great Red Spot, which is a giant storm larger than Earth. Jupiter has 79 known moons, including the four largest: Io, Europa, Ganymede, and Callisto, often referred to as the Galilean moons.
"""
with open("test_document.txt", "w") as f:
    f.write(document_content)

# Upload the test_document.txt file
file_path = "test_document.txt"

print("Uploading document...")
with open(file_path, "rb") as f:
    files = {'file': (os.path.basename(file_path), f, 'text/plain')}
    response = requests.post("http://localhost:8000/upload-document", files=files)

print(f"Upload Response: {response.json()}")

# Wait a bit for the vector store to be built and saved after upload
time.sleep(10)

# Test the /ask-rag endpoint
rag_question = "What is the largest planet in our solar system?"
response_rag = requests.post(
    "http://localhost:8000/ask-rag",
    json={"text": rag_question}
)
print(f"RAG Question: {rag_question}")
print(f"RAG Response: {response_rag.json()}")

# Test the /summarize-rag endpoint
summary_request = "Jupiter's moons"
response_summary = requests.post(
    "http://localhost:8000/summarize-rag",
    json={"text": summary_request}
)
print(f"Summarize Request: {summary_request}")
print(f"Summary Response: {response_summary.json()}")

FileNotFoundError: [Errno 2] No such file or directory: '/content/llm_api'

# Task
Successfully set up and test the RAG system, including the FastAPI server, document processing (creating and uploading `test_document.txt`), and vector store creation and interaction (asking RAG questions and requesting summaries).

## Ensure llm_api directory exists

### Subtask:
Add code to explicitly create the `/content/llm_api` directory if it does not already exist, before attempting to change the current working directory. This will prevent `FileNotFoundError`.


In [ ]:
import requests
import os
import time
import subprocess

# Ensure the /content/llm_api directory exists
if not os.path.exists('/content/llm_api'):
    os.makedirs('/content/llm_api')

# Ensure any previous Uvicorn processes are killed
subprocess.run("kill -9 $(lsof -t -i:8000) 2>/dev/null || true", shell=True)

# Change directory to where app.py is located
os.chdir('/content/llm_api')

# Restart Uvicorn server with the updated app.py
uvicorn_log_path = "uvicorn.log" # Log file will be in /content/llm_api
# Clear the log file before starting uvicorn to avoid reading old logs
open(uvicorn_log_path, 'w').close()
# Use subprocess.Popen for non-blocking execution of uvicorn
subprocess.Popen(f"nohup python -m uvicorn app:app --host 0.0.0.0 --port 8000 > {uvicorn_log_path} 2>&1 &", shell=True)

# Wait for the server to start up and models to load (polling log file)
print("Waiting for FastAPI server to start and models to load...")
server_and_models_ready = False
start_time = time.time()
timeout = 300 # Increased timeout to 5 minutes for model loading

while time.time() - start_time < timeout:
    if os.path.exists(uvicorn_log_path):
        with open(uvicorn_log_path, 'r') as f:
            log_content = f.read()
            if "✅ Models loaded" in log_content:
                server_and_models_ready = True
                print("FastAPI server and models are ready.")
                break
    time.sleep(5) # Check every 5 seconds

if not server_and_models_ready:
    # Read the log file one last time if not ready, to include it in the error message
    final_log_content = "Log file not found or empty." if not os.path.exists(uvicorn_log_path) else open(uvicorn_log_path, 'r').read()
    raise Exception(f"FastAPI server and models did not start in time ({timeout} seconds). Check {uvicorn_log_path} for errors. \n--- Uvicorn Log Content ---\n{final_log_content}\n---------------------------")

# Create a dummy text file named test_document.txt
document_content = """
Jupiter is the largest planet in our solar system. It is a gas giant with a mass more than two and a half times that of all the other planets in the Solar System combined, but slightly less than one-thousandth the mass of the Sun. Jupiter is the fifth planet from the Sun. It is primarily composed of hydrogen and helium. It has a Great Red Spot, which is a giant storm larger than Earth. Jupiter has 79 known moons, including the four largest: Io, Europa, Ganymede, and Callisto, often referred to as the Galilean moons.
"""
with open("test_document.txt", "w") as f:
    f.write(document_content)

# Upload the test_document.txt file
file_path = "test_document.txt"

print("Uploading document...")
with open(file_path, "rb") as f:
    files = {'file': (os.path.basename(file_path), f, 'text/plain')}
    response = requests.post("http://localhost:8000/upload-document", files=files)

print(f"Upload Response: {response.json()}")

# Wait a bit for the vector store to be built and saved after upload
time.sleep(10)

# Test the /ask-rag endpoint
rag_question = "What is the largest planet in our solar system?"
response_rag = requests.post(
    "http://localhost:8000/ask-rag",
    json={"text": rag_question}
)
print(f"RAG Question: {rag_question}")
print(f"RAG Response: {response_rag.json()}")

# Test the /summarize-rag endpoint
summary_request = "Jupiter's moons"
response_summary = requests.post(
    "http://localhost:8000/summarize-rag",
    json={"text": summary_request}
)
print(f"Summarize Request: {summary_request}")
print(f"Summary Response: {response_summary.json()}")

Waiting for FastAPI server to start and models to load...
